Title: 

Introduction:
1. Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
2. Clearly state the question you will try to answer with your project
3. Identify and describe the dataset that will be used to answer the question

Preliminary exploratory data analysis:
1. Demonstrate that the dataset can be read from the web into R 
2. Clean and wrangle your data into a tidy format
3. Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
4. Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.

Methods:

Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
Describe at least one way that you will visualize the results


Expected outcomes and significance:

What do you expect to find?
What impact could such findings have?
What future questions could this lead to?

In [8]:
library(tidyverse)
library(tidymodels)

# start the coding here

In [1]:
# read data here

cleveland_data <- read.table("data/heart_disease/processed.cleveland.data", sep=",")
colnames(cleveland_data)<-c('age','sex','cp','trestbps','chol','fbs', 'restcg', 'thalach',
                                     'exang','oldpeak','slope','ca','thal','num')
#switzerland_data

# clean the data
cleveland_clean <- cleveland_data |> mutate(age = as.numeric(age),
                                                sex = as_factor(sex),
                                                cp = as_factor(cp),
                                                trestbps = as.numeric(trestbps),
                                                chol = as.numeric(chol),
                                                fbs = as_factor(fbs),
                                                restcg = as_factor(restcg),
                                                thalach = as.numeric(thalach),
                                                exang = as_factor(exang),
                                                oldpeak = as.numeric(oldpeak),
                                                slope = as.numeric(slope),
                                                ca = as.numeric(ca),
                                                thal = as_factor(thal),
                                                num = as.numeric(num))
switzerland_clean

ERROR: Error in mutate(cleveland_data, age = as.numeric(age), sex = as_factor(sex), : could not find function "mutate"
